<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Diffusion_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Diffusion model

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-colorblind")

import math

import tensorflow as tf
from tensorflow.keras import(
    layers,
    models,
    optimizers,
    utils,
    callbacks,
    metrics,
    losses,
    activation,
)



#0. Parameters

In [ ]:
IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

#optimization
EMA = 0.999
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50

#1. Prepare the data

In [ ]:
#load the data
train_data = utils.image_dataset_from_directory(
    "/app/data/pytorch-challange-flower-dataset/dataset",
    labels = None,
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = None,
    shuffle = True,
    seed = 42,
    interpolation = "bilibear",
)

In [ ]:
#prepare the data
def preprocess(img):
  img = tf.cast(img, "float32") / 255.0
  return img

train = train_data.map(lambda x: preprocess(x))
train = train.repeat(DATASET_REPETITIONS)
train = train.batch(BATCH_SIZE, drop_remainder = True)

In [ ]:
#Show some items of clothing from the training set
train_sample = sample_batch(train)
display(train_sample)

#1.1 Diffusion schedules

In [ ]:
def linear_diffusion_schedule(diffusion_times):
  min_rate = 0.0001
  max_rate = 0.02
  betas = min_rate + diffusion_times_ * (max_rate - min_rate)
  alphas = 1 - betas
  alpha_bars = tf.math.cumprod(alphas)
  signal_rate = tf.sqrt(alpha_bars)
  noise_rate = tf.sqrt(1 - alpha_bars)
  return noise_rates  , signal_rates

In [ ]:
def cosine_diffusion_schedule(diffsuion_times):
  signal_rate = tf.cos(diffusion_times * math.pi / 2)
  noise_rates = tf.sin(diffusion_times * math.pi / 2)
  return noise_rates , signal_rates


In [ ]:
def offset_cosine_diffusion_schedule(diffusion_times):
  min_signal_rate = 0.02
  max_signal_rate = 0.95
  start_angle = tf.acos(max_signal_rate)
  end_angle = tf.acos(min_signal_rate)

  diffusion_angles = start_angle + diffusion_times * (end_angle -start_angle)

  signal_rates = tf.cos(diffusion_angles)
  noise_rates = tf.sin(diffusion_angles)

  return noise_rates , signal_rates

In [ ]:
T = 1000
diffusion_times = tf.convert_to_tensor([x /T for x in range(T)])
linear_noise_rates , linear_signal_rates = linear_diffusion_schedule(
    diffusion_times
)
cosine_noise_rates , cosine_signal_rates = cosine_diffusion_schedule(
    diffusion_times
)
(
    offset_cosine_noise_rates ,
    offset_cosine_signal_rates ,
) = offset_cosine_diffusion_schedule(diffusion_times)

In [ ]:
plt.plot(
    diffusion_times , linear_signal_rates ** 2 , linewidth = 1.5, label = "linear"
)
plt.plot(
    diffusion_times , cosine_signal_rates ** 2 , linewidth = 1.5, label = "cosine"
)
plt.plot(
    diffusion_times ,
    offset_cosine_signal_rates ** 2,
    linewidth = 1.5,
    label = "offset_cosine",
)

plt.xlabel("t/T" , fontsize = 12)
plt.ylabel(r"$\bar{\alpha_t}$ (signal)", fontsize = 12)
plt.legend()
plt.show()

In [ ]:
plt.plot(
    difffusion_times , linear_noise_rates ** 2, linearwidth = 1.5, label = "linear"
)
plt.plot(
    diffusion_times , cosine_noise_rates ** 2, linewidth = 1.5 , label = "cosine"
)
plt.plot(
    diffusion_times ,
    offset_cosine_noise_rates ** 2,
    linewidth = 1.5,
    label = "offset_cosine",
)

plt.xlabel("t/T" , fontsize = 12)
plt.ylabel(r"$1 -\bar{\alpha_t}$ (noise)" , fontsize = 12)
plt.legend()
plt.show()

#2. Build the model

In [ ]:
def sinusoidal_embedding(x):
  frequencies = tf.exp(
      tf.linspace(
          tf.math.log(1.0),
          tf.math.log(1000, 0),
          NOISE_EMBEDDING_SIZE // 2,
      )
  )
  angular_speeds = 2.0 * math.pi * frequencies
  embeddings = tf.concat(
      [tf.sin(angular_speeds * x) , tf.cos(angular_speeds * x)], axis = 3
  )
  return embeddings


In [ ]:
embedding_list = []
for y in np.arange(0,1,0.01):
  embedding_list.append(sinusoidal_embedding(np.array([[[[y]]]]))[0][0][0])
embedding_array = np.array(np.transpose(embedding_list))
fig, ax = plt.subplots()
ax.set_xticks(
    np.arange(0, 100 , 10), labels = np.round(np.arange(0.0, 1.0, 0.1), 1)
)
ax.set_ylabel("embedding dimension" , fontsize = 8)
ax.set_xlabel("noise variance" , fontsize = 8)
plt.pcolor(embedding_array , cmap= "coolwarm")
plt.colorbar(orientation = "horizontal" , label = "embedding value")
ax.imshow(embedding_array, inerpolation = "nearest" , origin = "lower")
plt.shoe()


In [ ]:
def ResidualBlock(width):
  def apply(x):
    input_width = x.shape[3]
    if input_width == width:
      residual = x
    else:
      residual = layers.Conv2D(width, kernel_size = 1)(x)
    x = layers.BatchNormalization(center = False, scale = False)(x)
    x = layers.Conv2D(
        width , kernel_size = 3, padding = "same" , activation = activations.swish
    )(x)
    x = layers.Conv2D(width , kernel_size = 3 , padding = "same")(x)
    x = layers.Add()([x, residual])
    return x
  return apply


def DownBlock(width , block_depth):
  def apply(x):
    x , skips = x
    for _ in range(block_depth):
      x = ResidualBlock(width)(x)
      skips.append(x)
    x = layers.AveragePooling2D(pool_size = 2)(x)
    return x
  return apply

def UpBlock(width , block_depth):
  def apply(x):
    x , skips = x
    x = layers.UpSampling2D(size = 2, interpolation = "bilinear")(x)
    for _ in range(block_depth):
      x = layers.Concatenate()([x, skips.pop()])
      x = ResidualBlock(width)(x)
    return x

  return apply



In [ ]:
#Build the U- NET

noisy_images = layers.Input(shape = (IMAGE_SIZE, IMAGE_SIZE, 3))
x = layers.Conv2D(32, kernel_size = 1)(noisy_images)

noise_varaiance = layers.Input(shape = (1, 1, 1))
noise_embedding = layers.Lambda(sinusoidal_embedding)(noise_varaiance)
noise_embedding = layers.UpSampling2D(size = IMAGE_SIZE , interpolation = "nearest")(
    noise_embedding
)

x = layers.Concatenate()([x, noise_embedding])

skips = []

x = DownBlock(32, block_depth = 2)([x, skips])
x = DownBlock(64, block_depth = 2)([x , skips])
x = DownBlock(96, block_depth = 2)([x , skips])

x = ResidualBlock(128)(x)
x = ResidualBlock(128)(x)

x = UpBlock(96, block_depth = 2)([x , skips])
x = UpBlock(64, block_depth = 2)([x , skips])
x = UpBlock(32, block_depth = 2)([x , skips])

x = layers.Conv2D(3, kernel_size = 1, kernel_initializer = "zeros")(x)

unet = models.Model([noisy_images , noise_varaiance] , x , name="unet")

In [ ]:
class